In [1]:
import torch
import gymnasium as gym
import pygame
import numpy as np
from pendulum import PendulumEnv

In [3]:
#env = gym.make('Pendulum-v1', g=9.81, render_mode='human', max_episode_steps= 1000)
env = PendulumEnv(g=9.81, render_mode='human')
observation, info = env.reset(options={'x_init':0.1, 'y_init':0.1})


pygame.font.init() 
text_font = pygame.font.SysFont('Comic Sans MS', 18)


env.screen = pygame.display.set_mode((env.screen_dim, env.screen_dim + 100))

bottom_surf = pygame.Surface((env.screen_dim, 100))
bottom_surf.fill((155, 155, 155))
env.screen.blit(bottom_surf, (0, env.screen_dim))

help_text = text_font.render('reset(r), random (a), no-action (n), swing- (s), quit(Esc)', False,(0,0,0))
#env.screen.blit(help_text, (20,env.screen_dim+60))

running = True 
action_type = "random_action" 

t = 0
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            runnig = False
            break
        
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
                break
            if event.key == pygame.K_a:
                action_type = "random_action" 
            if event.key == pygame.K_n:
                action_type = "no_action"
            if event.key == pygame.K_c:
                action_type = "const_action"
            if event.key == pygame.K_s:
                action_type = "swing_up_action"
            if event.key == pygame.K_r:
                observation, info = env.reset(options={'x_init':0.1, 'y_init':0.1})
                t = 0 

    t = t + 1
    if action_type == "random_action":
        action = env.action_space.sample()  # agent policy that uses the observation and info
    if action_type == "no_action": 
        action = [0]
    if action_type == "const_action":
        action = [2.0]
    if action_type == "swing_up_action":
        k1 = 10
        k2 = 2
        feedback_u_val = -k1*observation[1] - k2*observation[2] 
        #sin_theta_max = 1./k
        #cos_theta_max = np.sqrt(1-sin_theta_max*sin_theta_max)
        if observation[0] > 0:
            action = [feedback_u_val]
        else:
            if observation[2]<5:
                if observation[2]>0:
                    action = [2.0]
                else:
                    action = [-2.0]
            else:
                action = [0]
                

    observation, reward, terminated, truncated, info = env.step(action)

    time_text = text_font.render('time %.1f' %(t*env.dt), False,(0,0,0))
    env.screen.blit(bottom_surf, (0, env.screen_dim))
    env.screen.blit(time_text, (200,env.screen_dim+20))
    env.screen.blit(help_text, (20,env.screen_dim+60))
    #pygame.display.flip()

    if terminated or truncated:
        observation, info = env.reset(options={'x_init':0.01, 'y_init':0})
        t = 0 
    

env.reset()
env.close()
